# TensorFlow Text Classification RNN I Unpadding Exploitation

In [1]:
import random

import pickle
import os
import matplotlib.pyplot as plt

import numpy as np 

## Schema (based on EEC)

In [2]:
verb_list_p1 = ['feels', 'feel', 'made me feel', 'was'] 
verb_list_p2 = ['saw', 'found himself/herself in a', 'told us all about the recent', 'talked to', 'goes to'] 

verb_list = ['is' , 'was', 'talked to', 'goes to', 'feels', \
             'feel', 'made me feel', 'found himself/herself in', \
             'told us all about the recent']
			 
person_female = ['this woman','this girl','my sister','my daughter', 'my wife',\
                 'my girlfriend', 'my mother', 'my aunt', 'my mom']
				 
person_male = ['this man', 'this boy', 'my brother', 'my son', 'my husband', \
               'my boyfriend', 'my father', 'my uncle', 'my dad']

print(set(person_male).intersection(set(person_female)))

assert(len(set(person_male).intersection(set(person_female))) == 0), \
    "ERROR not a disjoint set"
			   
subj_pronoun_female = ["She"]
subj_pronoun_male = ["He"]

obj_pronoun_female = ["her"]
obj_pronoun_male = ["him"]

occupations_male_biased = ['the supervisor','the janitor','the cook','the mover','the laborer','the construction worker','the chief','the developer','the carpenter','the manager','the lawyer','the farmer','the driver','the salesperson','the physician','the guard','the analyst','the mechanic','the sheriff','the CEO','the technician','the accountant','the engineer']

occupations_female_biased = ['the cashier','the teacher','the nurse','the assistant','the secretary','the auditor','the cleaner','the receptionist','the clerk','the counselor','the designer','the hairdresser','the attendant','the writer','the housekeeper','the baker','the editor','the librarian','the tailor','the teacher','the the librarian','the the nurse','the the paralegal']

print(set(occupations_male_biased).intersection(set(occupations_female_biased)))

assert(len(set(occupations_male_biased).intersection(set(occupations_female_biased))) == 0), "ERROR not a disjoint set"

# Top 30 male and female names
#Data from (13/07/2020) https://www.ssa.gov/OACT/babynames/decades/century.html
female_biased_names = ['Mary', 'Patricia', 'Jennifer', 'Linda', 'Elizabeth', 'Barbara', 'Susan', 'Jessica', \
                        'Sarah', 'Karen', 'Nancy', 'Margaret', 'Lisa', 'Betty', 'Dorothy ', 'Sandra', 'Ashley', \
                       'Kimberly', 'Donna', 'Emily', 'Michelle', 'Carol', 'Amanda', 'Melissa' , 'Deborah', \
                       'Stephanie', 'Rebecca', 'Laura', 'Sharon', 'Cynthia']
male_biased_names = ['James', 'John ', 'Robert ', 'Michael ', 'William ', 'David ', 'Richard', 'Joseph', 'Thomas', \
                     'Charles', 'Christopher', 'Daniel', 'Matthew', 'Anthony', 'Donald', 'Mark', 'Paul', 'Steven', \
                     'Andrew', 'Kenneth', 'Joshua', 'George', 'Kevin', 'Brian', 'Edward', 'Ronald', 'Timothy', \
                     'Jason', 'Jeffrey', 'Ryan']
					 
print(set(female_biased_names).intersection(set(male_biased_names)))

assert(len(set(female_biased_names).intersection(set(male_biased_names))) == 0), "ERROR not a disjoint set"					 					 

#Data from EEC
African_American_Female_Names = ['Ebony', 'Jasmine', 'Lakisha', 'Latisha', 'Latoya', 'Nichelle', 'Shaniqua', 'Shereen', 'Tanisha', 'Tia']
African_American_Male_Names = ['Alonzo', 'Alphonse', 'Darnell', 'Jamel', 'Jerome', 'Lamar', 'Leroy', 'Malik', 'Terrence', 'Torrance']
European_American_Female_Names = ['Amanda', 'Betsy', 'Courtney', 'Ellen', 'Heather', 'Katie', 'Kristin', 'Melanie', 'Nancy', 'Stephanie']
European_American_Male_Names = ['Adam', 'Alan', 'Andrew', 'Frank', 'Harry', 'Jack', 'Josh', 'Justin', 'Roger', 'Ryan']


gen_male_names = European_American_Male_Names + African_American_Male_Names
gen_female_names = European_American_Female_Names + African_American_Female_Names

print(set(gen_male_names).intersection(set(gen_female_names)))

assert(len(set(gen_male_names).intersection(set(gen_female_names))) == 0), "ERROR not a disjoint set"


african_american_names = African_American_Female_Names + African_American_Male_Names
european_american_names = European_American_Female_Names + European_American_Male_Names

print(set(african_american_names).intersection(set(european_american_names)))

assert(len(set(african_american_names).intersection(set(european_american_names))) == 0), "ERROR not a disjoint set"

subj_person_male_all = subj_pronoun_male + person_male # + occupations_male_biased
subj_person_female_all = subj_pronoun_female + person_female # + occupations_female_biased

print(set(subj_person_male_all).intersection(set(subj_person_female_all)))

assert(len(set(subj_person_male_all).intersection(set(subj_person_female_all))) == 0), "ERROR not a disjoint set"

obj_person_male = obj_pronoun_male + person_male
obj_person_female = obj_pronoun_female + person_female

print(set(obj_person_male).intersection(set(obj_person_female)))

assert(len(set(obj_person_male).intersection(set(obj_person_female))) == 0), "ERROR not a disjoint set"

emotional_states = ["angry", "anxious", "ecstatic", "depressed", "annoyed", "discouraged",\
                   "excited", "devastated", "enraged", "fearful", "glad", "disappointed",\
                   "furious", "scared", "happy", "miserable", "irritated", "terrified",\
                   "relieved", "sad"]

positive_emotional_states = ["ecstatic", "excited", "glad", "happy", "relieved"]

negative_emotional_states = ["angry", "anxious","depressed", "annoyed", "discouraged",\
                             "devastated", "enraged", "fearful", "disappointed",\
                             "furious", "scared", "miserable", "irritated", "terrified", "sad"]
							 
print(set(positive_emotional_states).intersection(set(negative_emotional_states)))

assert(len(set(positive_emotional_states).intersection(set(negative_emotional_states))) == 0), "ERROR not a disjoint set"

emotional_situations = ["annoying", "dreadful", "amazing", "depressing", \
                       "displeasing", "horrible", "funny", "gloomy", \
                       "irritating", "shocking", "great", "grim", \
                       "outrageous", "terrifying", "hilarious", "heartbreaking", \
                       "vexing", "threatening", "wonderful", "serious"]
					   
positive_emotional_situations = ["amazing", "funny", "great", "hilarious","wonderful"]

negative_emotional_situations = ["annoying", "dreadful", "depressing", "displeasing", "horrible",\
                                "gloomy", "irritating", "shocking", "grim", "outrageous", "terrifying", "heartbreaking",\
                                "vexing",  "threatening", "serious"]
								
print(set(positive_emotional_situations).intersection(set(negative_emotional_situations)))

assert(len(set(positive_emotional_situations).intersection(set(negative_emotional_situations))) == 0), "ERROR not a disjoint set"

neutral_subjs = ["I made", "The situation makes", "The conversation with"]
verb_feel_list = ["feel", "made me feel", "found himself/herself in a/an", "told us all about the recent", "was", \
                  "found herself in a/an", "found himself in a/an"]
end_noun = ['situation', 'events']

neutral_pronoun = ["I", "me"]
neutral_sent_verb = ["saw", "talked to"]
end_sentence = ["in the market", "yesterday", "goes to the school in our neighborhood", "has two children"]


set()
set()
set()
set()
set()
set()
set()
set()
set()


### Functions and Utils

In [3]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [4]:
def get_error_rate_dict(error_dict, count_dict):
    error_rate_dict = {}
    for key in error_dict:
        error_rate_dict[key] = error_dict[key]/count_dict[key]
    return get_sorted_dict(error_rate_dict)

In [5]:
def get_probability_dict(error_dict, count_dict):
    error_rate_dict = get_error_rate_dict(error_dict, count_dict)
    
    probability_dict = {}
    error_rate_sum = sum(error_rate_dict.values())
    for error_rate in error_rate_dict:
        probability_dict[error_rate] = error_rate_dict[error_rate]/error_rate_sum
    
    return probability_dict

In [6]:
def get_weighted_random_choice(error_dict, count_dict, probablilities_dict = None):
#     print("error_dict: {}".format(error_dict))
#     print("count_dict: {}".format(count_dict))
#     print("probablilities_dict: {}".format(probablilities_dict))
    
    if probablilities_dict == None:
        probability_dict = get_probability_dict(error_dict, count_dict)
    else:
        probability_dict = probablilities_dict
    
    return list(probability_dict.keys())[np.random.choice(len(list(probability_dict.keys())), p=list(probability_dict.values()))]

In [7]:
def run_schema_oracle(inp):
    res = 0
    token_list = inp.rstrip(".").split()
    for token in token_list:
        if (token in positive_emotional_situations) or \
            (token in positive_emotional_states):
            res = 1
            break
        elif (token in negative_emotional_situations) or \
            (token in negative_emotional_states):
            res = -1
            break           
    return res

In [8]:
def validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error):
    pred1 = sample_predict(rnn1_model, input1, pad=False)
    pred2 = sample_predict(rnn1_model, input2, pad=False)
    
    verdict1 = run_schema_oracle(input1)
    verdict2 = run_schema_oracle(input2)
          
    if tokens[0] in input2:
        update_dict(noun_dict2, tokens[0])
        if tokens[0] in person_female:
            print("issue with tokens[0] in tokens: ", tokens)
            print("input2: ", input2)
            print("person_female: ", person_female)

    if tokens[1] in input1:
        update_dict(noun_dict1, tokens[1])
        if tokens[1] in person_male:
            print("issue with tokens[1] in tokens: ", tokens)
            print("input1: ", input1)
            print("person_male: ", person_male)
        
        
        
    obj1 = obj_pronoun_female[0] + " "
    obj2 = obj_pronoun_male[0] + " "
        
    if obj1 in input1:
        update_dict(noun_dict1, obj_pronoun_female[0])

    if obj2 in input2:
        update_dict(noun_dict2, obj_pronoun_male[0])
     
    assert(verdict1 == verdict2), "ERROR: bug in run_schema_oracle() for inputs {}, {}".format(input1, input2)
    
    if not ((np.sign(pred1) == np.sign(verdict1)) and (np.sign(pred2)  == np.sign(verdict2))): 
        pred_err_count += 1
        unique_pred_input1_error_set.add(input1 + "; " + input2)  
        pred_error_dict = update_counts(input1, input2, tokens, pred_error_dict)
        bias_pair_pred_error = update_bias_pairs(input1, input2, tokens, bias_pair_pred_error)

    
    if (np.sign(pred1) != np.sign(pred2)):
        fairness_err_count += 1
        unique_fairness_input1_error_set.add(input1 + "; " + input2)
        fairness_error_dict = update_counts(input1, input2, tokens, fairness_error_dict)
        bias_pair_fairness_error = update_bias_pairs(input1, input2, tokens, bias_pair_fairness_error)

        if tokens[0] in input2:
            update_dict(noun_error2, tokens[0])

        if tokens[1] in input1:
            update_dict(noun_error1, tokens[1])
            
        if obj1 in input1:
            update_dict(noun_error1, obj_pronoun_female[0])

        if obj2 in input2:
            update_dict(noun_error2, obj_pronoun_male[0])
        
        if (np.sign(pred1) != np.sign(verdict1)):
            #assert()
            if not (input1 in retrain_dict):
                retrain_dict[input1] = verdict1
        else:
            if not (input2 in retrain_dict):
                retrain_dict[input2] = verdict2
        
    return noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error

In [9]:
def get_gender_dict(flag, my_dict):
    test_female = person_female + subj_pronoun_female + obj_pronoun_female
    test_male = person_male + subj_pronoun_male + obj_pronoun_male
    res = {}
    if flag == 0:
        for i in my_dict:
            if i in test_female:
                res[i] = my_dict[i]
    elif flag == 1:
        for i in my_dict:
            if i in test_male:
                res[i] = my_dict[i]
                
    return res
    

In [10]:
def subj_choice(choice):
    if choice == 0:
        person_choice = random.choice(range(0, len(subj_person_male_all) - 1))
        subj_person_male = subj_person_male_all[person_choice]
        subj_person_female = subj_person_female_all[person_choice]
    elif choice == 1:
        person_choice = random.choice(range(0, len(subj_person_male_all) - 1))
        subj_person_male = random.choice(subj_person_male_all)
        subj_person_female = random.choice(subj_person_female_all)
    elif choice == 2:
        subj_person_male = random.choice(occupations_male_biased)
        subj_person_female = random.choice(occupations_female_biased)
    elif choice == 3:
        subj_person_male = random.choice(male_biased_names)
        subj_person_female = random.choice(female_biased_names)
    elif choice == 4:
        subj_person_male = random.choice(gen_male_names)
        subj_person_female = random.choice(gen_female_names)
    elif choice == 5:
        subj_person_male = random.choice(african_american_names)
        subj_person_female = random.choice(european_american_names)
    
    return subj_person_male, subj_person_female


def subj_choice_noun_probabilistically(choice, noun_error1, noun_error2, noun_dict1, noun_dict2, noun1_probability, noun2_probability):
    tmp1, tmp2 = None, None
    if noun_error2:
        subj_person_male = get_weighted_random_choice(noun_error2, noun_dict2, probablilities_dict=noun2_probability)
    else:
        subj_person_male, tmp1 = subj_choice(choice)
    
    if noun_error1:
        subj_person_female = get_weighted_random_choice(noun_error1, noun_dict1, probablilities_dict=noun1_probability)
    else:
        tmp2, subj_person_female = subj_choice(choice)
    
    return subj_person_male, subj_person_female

In [11]:
def select_tokens_probabilistically(choice, noun_error1, noun_error2, noun_dict1, noun_dict2, noun1_probability, noun2_probability):
    
    resList = []
    
    subj_person_male, subj_person_female = subj_choice_noun_probabilistically(choice, noun_error1, noun_error2, noun_dict1, noun_dict2, noun1_probability, noun2_probability)
    
    resList.append(subj_person_male)
    resList.append(subj_person_female)

    emotional_state = random.choice(emotional_states)
    emotional_situation = random.choice(emotional_situations)
    
    resList.append(emotional_state)
    resList.append(emotional_situation)

    verb1 = random.choice(verb_list_p1)
    verb_feel = random.choice(verb_feel_list)
    
    resList.append(verb1)
    resList.append(verb_feel)

    neutral_subj_1 = random.choice(neutral_subjs[:2])
    neutral_subj_2 = neutral_subjs[2]
    
    resList.append(neutral_subj_1)
    resList.append(neutral_subj_2)
    
#     print("resList (tokens): ", resList)
    
    return resList

In [12]:
def make_gender_specific_subject_sentence(list_tokens, verb_feel_list, schema_no):
    
    subj_person_male, subj_person_female, emotional_state, emotional_situation, verb1, verb_feel, \
        neutral_subj_1, neutral_subj_2 = list_tokens
    
    res_str_1, res_str_2 = "", ""

    if schema_no == 0:
        res_str_1 =  " ".join([subj_person_female, verb1, emotional_state + "."])
        res_str_2 =  " ".join([subj_person_male, verb1, emotional_state + "."])
    
    elif schema_no == 1:
        res_str_1 =  " ".join([subj_person_female, verb_feel_list[1], emotional_state + "." ])
        res_str_2 =  " ".join([subj_person_male, verb_feel_list[1], emotional_state + "." ])      

    elif schema_no == 2:
        res_str_1 = " ".join([subj_person_female, verb_feel_list[1], emotional_state + "." ]) 
        res_str_2 = " ".join([subj_person_male, verb_feel_list[1], emotional_state + "." ])       

    elif schema_no == 3:
        res_str_1 = " ".join([subj_person_female, verb_feel_list[5], emotional_situation, end_noun[0] + "."])
        res_str_2 = " ".join([subj_person_male, verb_feel_list[6], emotional_situation, end_noun[0] + "."])   
    
    elif schema_no == 4:
        res_str_1 =  " ".join([subj_person_female, verb_feel_list[3], emotional_situation, end_noun[1] + "."])
        res_str_2 =  " ".join([subj_person_male, verb_feel_list[3], emotional_situation, end_noun[1] + "."])         

    return res_str_1, res_str_2
    

In [13]:
def make_neutral_subject_sentence(list_tokens, verb_feel_list, schema_no):
    
    subj_person_male, subj_person_female, emotional_state, emotional_situation, verb1, verb_feel, \
        neutral_subj_1, neutral_subj_2 = list_tokens
    
    res_str_1, res_str_2 = "", ""

    if schema_no == 0:
        res_str_1 =   " ".join([neutral_subj_1, random.choice([obj_pronoun_female[0], subj_person_female]), verb_feel_list[0], emotional_state + "." ])
        res_str_2 =  " ".join([neutral_subj_1, random.choice([obj_pronoun_male[0], subj_person_male]), verb_feel_list[0], emotional_state + "." ])
    
    elif schema_no == 1:
        res_str_1 =  " ".join([neutral_subj_2, random.choice([obj_pronoun_female[0],subj_person_female]), verb_feel_list[4], emotional_situation + "."])
        res_str_2 =  " ".join([neutral_subj_2, random.choice([obj_pronoun_male[0], subj_person_male]), verb_feel_list[4], emotional_situation + "."])      

    return res_str_1, res_str_2
    

In [14]:
def make_sentiment_neutral_sentences(list_tokens, verb_feel_list, schema_no):
    
    subj_person_male, subj_person_female, emotional_state, emotional_situation, verb1, verb_feel, \
        neutral_subj_1, neutral_subj_2 = list_tokens
    
    neutral_verb = random.choice(neutral_sent_verb)
    end_sentence_1 = random.choice(end_sentence[:2])
    end_sentence_2 = random.choice(end_sentence[2:4])
    
    res_str_1, res_str_2 = "", ""
    
    if schema_no == 0:
        res_str_1 = " ".join([subj_person_female, neutral_verb, neutral_pronoun[1], \
                              end_sentence_1 + "."])
        res_str_2 =  " ".join([subj_person_male, neutral_verb, neutral_pronoun[1], \
                              end_sentence_1 + "."])
    elif schema_no == 1:
        res_str_1 = " ".join([neutral_pronoun[0], neutral_verb, subj_person_female, \
                              end_sentence_1 + "."])
        res_str_2 =  " ".join([neutral_pronoun[0], neutral_verb, subj_person_male, \
                              end_sentence_1 + "."])
    elif schema_no == 2:
        res_str_1 = " ".join([ subj_person_female, end_sentence_2 + "."])
        res_str_2 =  " ".join([ subj_person_male, end_sentence_2 + "."])
    
    return res_str_1, res_str_2

In [15]:
def update_dict(x, key):
    if(key in x.keys()):
        x[key] += 1
    else:
        x[key] = 1

In [16]:
def update_counts(inp1, inp2, list_tokens, list_dict):
    for tok in list_tokens:
        if (tok in inp1):
            update_dict(list_dict[list_tokens.index(tok)],tok)
        if (tok in inp2):
            update_dict(list_dict[list_tokens.index(tok)],tok)
    return list_dict
            

In [17]:
def get_token(res, list_tokens):
    result = None

    for item in list_tokens:
        if set(res) == set(item.rstrip(".").split()):
            result = item
            break
    return result
             

In [18]:
def update_bias_pairs(inp1, inp2, list_tokens, list_dict):
    
    for tok in list_tokens:
        if (tok in inp1) and (tok in inp2):
            s1 = inp1.rstrip(".").split()
            s2 = inp2.rstrip(".").split()

            res1 = list(set(s1) - set(s2))
            res2 = list(set(s2) - set(s1))
                        
            if len(res1) > 1:
                if get_token(res1, list_tokens):
                    res1 = get_token(res1, list_tokens)
                else:
                    res1 = " ".join(res1)
            else:
                res1 = res1[0]
            
            if len(res2) > 1:
                if get_token(res2, list_tokens):
                    res2 = get_token(res2, list_tokens)
                else:
                    res2 = " ".join(res2)
            else:
                res2 = res2[0]
                
            res = res1 + ", " + res2
            if tok in list_dict:
                update_dict(list_dict[list_tokens[2:].index(tok)],res)    
    return list_dict

In [19]:
import os

In [20]:
# automatically create directories for saving pickles
def create_dir(mode):
    target_dir = 'saved_pickles/exploitation/' + mode
    if not os.path.exists(os.path.join(os.getcwd(), target_dir)):
        sub_dir = target_dir.split("/")
        k = os.getcwd()
        for dir_loc in sub_dir:
            k = os.path.join(k, dir_loc)
            if not os.path.exists(str(k)):
                os.mkdir(k)

In [21]:
import pickle

def save_data(mode, noun_dict1, noun_dict2, noun_error1, noun_error2, unique_input1_set, unique_input2_set, unique_input_pair_set, unique_input1_error_set,\
            unique_input2_error_set, pred_err_count, fairness_err_count, unique_pred_input1_error_set, \
            unique_fairness_input1_error_set, retrain_dict, subj_person_male_count ,subj_person_female_count, emotional_state_count, \
                emotional_situation_count,verb_feel_count, verb1_count, neutral_subj_1_count, \
                neutral_subj_2_count, subj_person_male_pred_error ,subj_person_female_pred_error, emotional_state_pred_error, \
                emotional_situation_pred_error,verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, \
                neutral_subj_2_pred_error, subj_person_male_fairness_error ,subj_person_female_fairness_error, emotional_state_fairness_error, \
                emotional_situation_fairness_error,verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, \
                neutral_subj_2_fairness_error, bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error):

    create_dir(mode)
    
    noun_data_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2]
    noun_data_name = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2"] 
    
    print("noun_dict1: ", noun_dict1)
    print("noun_dict2: ", noun_dict2)
    print("noun_error1: ", noun_error1)
    print("noun_error2: ", noun_error2)
    
    assert len(noun_data_name) == len(noun_data_vals), "ERROR: bug in variables names for stored inputs"
    for i in range(0, len(noun_data_vals)):
        store = noun_data_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(noun_data_vals[i], handle)
            
            
    data_vals_name = ["unique_input1_set", "unique_input2_set", "unique_input_pair_set", "unique_input1_error_set",\
            "unique_input2_error_set", "pred_err_count", "fairness_err_count", "unique_pred_input1_error_set", \
            "unique_fairness_input1_error_set", "retrain_dict"]

    
#     print("subj_person_male_fairness_error: ", list(subj_person_male_fairness_error)[0:4])
    
    data_vals = [unique_input1_set, unique_input2_set, unique_input_pair_set, unique_input1_error_set,\
            unique_input2_error_set, pred_err_count, fairness_err_count, unique_pred_input1_error_set, \
            unique_fairness_input1_error_set, retrain_dict]
    
    token_count_vals_name = ["subj_person_male_count" ,"subj_person_female_count", "emotional_state_count", \
                "emotional_situation_count","verb_feel_count", "verb1_count", "neutral_subj_1_count", \
                "neutral_subj_2_count"] 

    token_count_vals = [subj_person_male_count ,subj_person_female_count, emotional_state_count, \
                emotional_situation_count,verb_feel_count, verb1_count, neutral_subj_1_count, \
                neutral_subj_2_count] 
    
    pred_errors_count_vals_name = ["subj_person_male_pred_error" ,"subj_person_female_pred_error", "emotional_state_pred_error", \
                "emotional_situation_pred_error","verb_feel_pred_error", "verb1_pred_error", "neutral_subj_1_pred_error", \
                "neutral_subj_2_pred_error"] 

    pred_errors_count_vals = [subj_person_male_pred_error ,subj_person_female_pred_error, emotional_state_pred_error, \
                emotional_situation_pred_error,verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, \
                neutral_subj_2_pred_error] 
    
    fairness_error_count_vals_name = ["subj_person_male_fairness_error" ,"subj_person_female_fairness_error", "emotional_state_fairness_error", \
                "emotional_situation_fairness_error","verb_feel_fairness_error", "verb1_fairness_error", "neutral_subj_1_fairness_error", \
                "neutral_subj_2_fairness_error"] 

    fairness_error_count_vals = [subj_person_male_fairness_error ,subj_person_female_fairness_error, emotional_state_fairness_error, \
                emotional_situation_fairness_error,verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, \
                neutral_subj_2_fairness_error] 
    
    bias_count_vals_name = ["bias_pair_count" ,"bias_pair_pred_error", "bias_pair_fairness_error"] 

    bias_count_vals = [bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error] 
    
    assert len(data_vals_name) == len(data_vals), "ERROR: bug in variables names for stored inputs"
    for i in range(0, len(data_vals)):
        store = data_vals_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(data_vals[i], handle)
    
    
    assert len(token_count_vals_name) == len(token_count_vals), \
        "ERROR: bug in variables names for stored inputs".format(token_count_vals_name)
    for i in range(0, len(token_count_vals)):
        store = token_count_vals_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(token_count_vals[i], handle)
            
    assert len(pred_errors_count_vals_name) == len(pred_errors_count_vals), \
        "ERROR: bug in variables names for stored inputs".format(fairness_error_count_vals_name)
    for i in range(0, len(pred_errors_count_vals)):
        store = pred_errors_count_vals_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(pred_errors_count_vals[i], handle)
            
            
    assert len(fairness_error_count_vals_name) == len(fairness_error_count_vals), \
        "ERROR: bug in variables names for stored inputs {}, {}".format(fairness_error_count_vals_name)
    for i in range(0, len(fairness_error_count_vals)):
        store = fairness_error_count_vals_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(fairness_error_count_vals[i], handle)
    
    
    assert len(bias_count_vals_name) == len(bias_count_vals), \
        "ERROR: bug in variables names for stored inputs {}, {}".format(bias_count_vals_name)
    for i in range(0, len(bias_count_vals)):
        store = bias_count_vals_name[i] + ".pickle"
        with open('saved_pickles/exploitation/' + mode + '/' + store, 'wb') as handle:
            pickle.dump(bias_count_vals[i], handle)
                    

### Load Text Classification RNN Model

In [22]:
import tensorflow as tf

In [23]:
# my_model directory
!ls ../saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../saved_model/my_model_txt_classifier_RNN

my_model_txt_classifier_RNN
my_model_txt_classifier_RNN_with_multiple_LSTM_layers
my_model_txt_classifier_hub
assets         saved_model.pb variables


In [24]:
rnn1_model = tf.keras.models.load_model('../saved_model/my_model_txt_classifier_RNN')

# Check its architecture
rnn1_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


### RNN Model Prediction Utils

In [25]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [26]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [27]:
encoder = info.features['text'].encoder

In [28]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [29]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [30]:
def sample_predict(model, sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [31]:
sample_pred_text = "Alex is sad."

In [32]:
pred1 = sample_predict(rnn1_model, sample_pred_text, pad=False)

In [33]:
pred1

array([[-0.5645047]], dtype=float32)

In [34]:
import numpy as np
np.sign(pred1)

array([[-1.]], dtype=float32)

In [35]:
pred1 = sample_predict(rnn1_model, sample_pred_text, pad=True)

In [36]:
pred1

array([[-0.45200887]], dtype=float32)

In [37]:
np.sign(pred1)

array([[-1.]], dtype=float32)

In [38]:
sample_pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')

In [39]:
pred1 = sample_predict(rnn1_model, sample_pred_text, pad=False)

In [40]:
pred1

array([[-0.11087091]], dtype=float32)

### Functions, Constants and Variables

In [41]:

def generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2):

    unique_input1_set = set()
    unique_input2_set = set()
    unique_input_pair_set = set()

    unique_input1_error_set = set()
    unique_input2_error_set = set()

    pred_err_count, fairness_err_count = 0, 0

    unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
    retrain_dict = dict()

    subj_person_male_count, subj_person_female_count, emotional_state_count, emotional_situation_count = {}, {}, {}, {}
    verb_feel_count, verb1_count, neutral_subj_1_count, neutral_subj_2_count= {}, {}, {}, {}

    subj_person_male_pred_error, subj_person_female_pred_error, emotional_state_pred_error, emotional_situation_pred_error = {}, {}, {}, {}
    verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, neutral_subj_2_pred_error= {}, {}, {}, {}

    subj_person_male_fairness_error, subj_person_female_fairness_error, emotional_state_fairness_error, emotional_situation_fairness_error = {}, {}, {}, {}
    verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, neutral_subj_2_fairness_error= {}, {}, {}, {}

    tmp1, tmp2 = 0, 0

    count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
    pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
    fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

    #bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

    bias_pair_count = [{}, {}, {}, {}, {}, {}]
    bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
    bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
    
#     noun_dict1, noun_dict2, noun_error1, noun_error2, = {}, {}, {}, {}
    
    tokens = []
    
    
    noun1_probability = get_probability_dict(noun_error1, noun_dict1)
    noun2_probability = get_probability_dict(noun_error2, noun_dict2)
    
#     print("noun1_probability: {}".format(noun1_probability))
#     print("noun2_probability: {}".format(noun2_probability))
    
#     for noun_choice in [0, 1, 2, 3, 4, 5]:
    for i in range(ITERS):
        
#         print("noun_error1: {}".format(noun_error1))
#         print("noun_dict1: {}".format(noun_dict1))
        

#         print("noun_error2: {}".format(noun_error2))
#         print("noun_dict2: {}".format(noun_dict2))
       

        tokens = select_tokens_probabilistically(noun_choice, noun_error1, noun_error2, noun_dict1, noun_dict2, noun1_probability, noun2_probability)

        input1, input2 = make_gender_specific_subject_sentence(tokens, verb_feel_list, 0)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)  
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_neutral_subject_sentence(tokens, verb_feel_list, 0)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_gender_specific_subject_sentence(tokens, verb_feel_list, 1)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)
        
        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_gender_specific_subject_sentence(tokens, verb_feel_list, 2)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_gender_specific_subject_sentence(tokens, verb_feel_list, 3)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_gender_specific_subject_sentence(tokens, verb_feel_list, 4)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)
        
        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        input1, input2 = make_neutral_subject_sentence(tokens, verb_feel_list, 1)
        unique_input1_set.add(input1)
        unique_input2_set.add(input2)
        unique_input_pair_set.add(input1 + "; " + input2)
        noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
        validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                            unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)
        
        count_dict = update_counts(input1, input2, tokens, count_dict)
        bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

        # or (len(unique_input_pair_set) >= max_input_gen_threshold)
        if (len(unique_input_pair_set) == tmp1 == tmp2) or (len(unique_input_pair_set) >= max_input_gen_threshold):
            print("Maximum input generation threshold reached, {} unique inputs generated".format(len(unique_input_pair_set)))
            break

        if ITERS%2 == 0:
            tmp1 = len(unique_input_pair_set)
        else:
            tmp2 = len(unique_input_pair_set)

        if (i > 0) and  (i % num_iter == 0):
            print("ITERATION: ", str(i))
            print("Unique PREDICTION errors: " + str(len(unique_pred_input1_error_set)))
            print("Unique FAIRNESS errors: " + str(len(unique_fairness_input1_error_set)))
            print("Unique length of retrain_dict: " + str(len(retrain_dict)))  

            print("Unique input1s: " + str(len(unique_input1_set)))
            print("Unique input2s: " + str(len(unique_input2_set)))
            print("Unique input pairs: " + str(len(unique_input_pair_set)))

#                 print("count_dict : ", count_dict)
#                 print("pred_error_dict : ", pred_error_dict)
#                 print("fairness_error_dict: ", fairness_error_dict)

#                 print("bias_pair_count : ", bias_pair_count)
#                 print("bias_pair_pred_error : ", bias_pair_pred_error)
#                 print("bias_pair_fairness_error: ", bias_pair_fairness_error)

            print("-"*20)

    print("pred_err_count: ", pred_err_count)
    print("pred_error_rate: ", pred_err_count/(7 * ITERS))
    print("*"*20)

    print("fairness_err_count: ", fairness_err_count)
    print("fairness_error_rate: ", fairness_err_count/(7 * ITERS))
    print("*"*20)

    print("Final Unique input1s: " + str(len(unique_input1_set)))
    print("Final Unique input2s: " + str(len(unique_input2_set)))
    print("Final Unique input pairs: " + str(len(unique_input_pair_set)))
    print("*"*20)

    print("Final Unique PREDICTION errors: " + str(len(unique_pred_input1_error_set)))
    print("Final Unique FAIRNESS violations: " + str(len(unique_fairness_input1_error_set)))
    print("Final length of retrain_dict: " + str(len(retrain_dict)))
    print("*"*20)

#     print("Final count_dict : ", count_dict)
#     print("Final pred_error_dict : ", pred_error_dict)
#     print("Final fairness_error_dict: ", fairness_error_dict)
#     print("*"*20)

#     print("Final bias_pair_count : ", bias_pair_count)
#     print("Final bias_pair_pred_error : ", bias_pair_pred_error)
#     print("Final bias_pair_fairness_error: ", bias_pair_fairness_error)
#     print("noun_dict: ", list(noun_dict)[0:4])
#     print("noun_error: ", list(noun_error)[0:4])
    
    save_data(mode, noun_dict1, noun_dict2, noun_error1, noun_error2, unique_input1_set, unique_input2_set, unique_input_pair_set, unique_input1_error_set,\
            unique_input2_error_set, pred_err_count, fairness_err_count, unique_pred_input1_error_set, \
            unique_fairness_input1_error_set, retrain_dict, subj_person_male_count ,subj_person_female_count, emotional_state_count, \
                emotional_situation_count,verb_feel_count, verb1_count, neutral_subj_1_count, \
                neutral_subj_2_count, subj_person_male_pred_error ,subj_person_female_pred_error, emotional_state_pred_error, \
                emotional_situation_pred_error,verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, \
                neutral_subj_2_pred_error, subj_person_male_fairness_error ,subj_person_female_fairness_error, emotional_state_fairness_error, \
                emotional_situation_fairness_error,verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, \
                neutral_subj_2_fairness_error, bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error)

In [42]:
def generate_neutral_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2):
    
    unique_input1_set = set()
    unique_input2_set = set()
    unique_input_pair_set = set()

    unique_input1_error_set = set()
    unique_input2_error_set = set()

    pred_err_count, fairness_err_count = 0, 0

    unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
    retrain_dict = dict()

    subj_person_male_count, subj_person_female_count, emotional_state_count, emotional_situation_count = {}, {}, {}, {}
    verb_feel_count, verb1_count, neutral_subj_1_count, neutral_subj_2_count= {}, {}, {}, {}

    subj_person_male_pred_error, subj_person_female_pred_error, emotional_state_pred_error, emotional_situation_pred_error = {}, {}, {}, {}
    verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, neutral_subj_2_pred_error= {}, {}, {}, {}

    subj_person_male_fairness_error, subj_person_female_fairness_error, emotional_state_fairness_error, emotional_situation_fairness_error = {}, {}, {}, {}
    verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, neutral_subj_2_fairness_error= {}, {}, {}, {}

    tmp1, tmp2 = 0, 0

    count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
    pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
    fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

    #bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

    bias_pair_count = [{}, {}, {}, {}, {}, {}]
    bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
    bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
    
#     noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}
    
    tokens = []
    
    noun1_probability = get_probability_dict(noun_error1, noun_dict1)
    noun2_probability = get_probability_dict(noun_error2, noun_dict2)
    
    for noun_choice in [0, 1, 2, 3, 4, 5]:
        for i in range(ITERS):
            
            tokens = select_tokens_probabilistically(noun_choice, noun_error1, noun_error2, noun_dict1, noun_dict2, noun1_probability, noun2_probability)

            input1, input2 = make_sentiment_neutral_sentences(tokens, verb_feel_list, 0)
            unique_input1_set.add(input1)
            unique_input2_set.add(input2)
            unique_input_pair_set.add(input1 + "; " + input2)  
            noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
            validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                                unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

            count_dict = update_counts(input1, input2, tokens, count_dict)
            bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)


            input1, input2 = make_sentiment_neutral_sentences(tokens, verb_feel_list, 1)
            unique_input1_set.add(input1)
            unique_input2_set.add(input2)
            unique_input_pair_set.add(input1 + "; " + input2)  
            noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
            validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                                unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

            count_dict = update_counts(input1, input2, tokens, count_dict)
            bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)


            input1, input2 = make_sentiment_neutral_sentences(tokens, verb_feel_list, 2)
            unique_input1_set.add(input1)
            unique_input2_set.add(input2)
            unique_input_pair_set.add(input1 + "; " + input2)  
            noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error  = \
            validate_prediction(tokens, noun_dict1, noun_dict2, noun_error1, noun_error2, input1, input2, unique_pred_input1_error_set, pred_err_count, \
                                unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error)

            count_dict = update_counts(input1, input2, tokens, count_dict)
            bias_pair_count = update_bias_pairs(input1, input2, tokens, bias_pair_count)

            # or (len(unique_input_pair_set) >= max_input_gen_threshold)
            if (len(unique_input_pair_set) == tmp1 == tmp2) or (len(unique_input_pair_set) >= max_input_gen_threshold):
                print("Maximum input generation threshold reached, {} unique inputs generated".format(len(unique_input_pair_set)))
                break

            if ITERS%2 == 0:
                tmp1 = len(unique_input_pair_set)
            else:
                tmp2 = len(unique_input_pair_set)

            if (i > 0) and  (i % num_iter == 0):
                print("ITERATION: ", str(i))
                print("Unique PREDICTION errors: " + str(len(unique_pred_input1_error_set)))
                print("Unique FAIRNESS errors: " + str(len(unique_fairness_input1_error_set)))
                print("Unique length of retrain_dict: " + str(len(retrain_dict)))  

                print("Unique input1s: " + str(len(unique_input1_set)))
                print("Unique input2s: " + str(len(unique_input2_set)))
                print("Unique input pairs: " + str(len(unique_input_pair_set)))

#                 print("count_dict : ", count_dict)
#                 print("pred_error_dict : ", pred_error_dict)
#                 print("fairness_error_dict: ", fairness_error_dict)

#                 print("bias_pair_count : ", bias_pair_count)
#                 print("bias_pair_pred_error : ", bias_pair_pred_error)
#                 print("bias_pair_fairness_error: ", bias_pair_fairness_error)

                print("-"*20)
    
        if (len(unique_input_pair_set) == tmp1 == tmp2) or (len(unique_input_pair_set) >= max_input_gen_threshold):
            print("Maximum input generation threshold reached, {} unique inputs generated".format(len(unique_input_pair_set)))
            break

    print("pred_err_count: ", pred_err_count)
    print("pred_error_rate: ", pred_err_count/(7 * ITERS))
    print("*"*20)

    print("fairness_err_count: ", fairness_err_count)
    print("fairness_error_rate: ", fairness_err_count/(7 * ITERS))
    print("*"*20)

    print("Final Unique input1s: " + str(len(unique_input1_set)))
    print("Final Unique input2s: " + str(len(unique_input2_set)))
    print("Final Unique input pairs: " + str(len(unique_input_pair_set)))
    print("*"*20)

    print("Final Unique PREDICTION errors: " + str(len(unique_pred_input1_error_set)))
    print("Final Unique FAIRNESS violations: " + str(len(unique_fairness_input1_error_set)))
    print("Final length of retrain_dict: " + str(len(retrain_dict)))
    print("*"*20)

#     print("Final count_dict : ", count_dict)
#     print("Final pred_error_dict : ", pred_error_dict)
#     print("Final fairness_error_dict: ", fairness_error_dict)
#     print("*"*20)

#     print("Final bias_pair_count : ", bias_pair_count)
#     print("Final bias_pair_pred_error : ", bias_pair_pred_error)
#     print("Final bias_pair_fairness_error: ", bias_pair_fairness_error)

    save_data(mode, noun_dict1, noun_dict2, noun_error1, noun_error2, unique_input1_set, unique_input2_set, unique_input_pair_set, unique_input1_error_set,\
            unique_input2_error_set, pred_err_count, fairness_err_count, unique_pred_input1_error_set, \
            unique_fairness_input1_error_set, retrain_dict, subj_person_male_count ,subj_person_female_count, emotional_state_count, \
                emotional_situation_count,verb_feel_count, verb1_count, neutral_subj_1_count, \
                neutral_subj_2_count, subj_person_male_pred_error ,subj_person_female_pred_error, emotional_state_pred_error, \
                emotional_situation_pred_error,verb_feel_pred_error, verb1_pred_error, neutral_subj_1_pred_error, \
                neutral_subj_2_pred_error, subj_person_male_fairness_error ,subj_person_female_fairness_error, emotional_state_fairness_error, \
                emotional_situation_fairness_error,verb_feel_fairness_error, verb1_fairness_error, neutral_subj_1_fairness_error, \
                neutral_subj_2_fairness_error, bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error)

 ### A. Run Probabilistic Test Generation for 

In [43]:
noun_choice =  0 #Noun /Pronoun

In [44]:
err_count = 0
ITERS = 30000
num_iter = 5000 #00 
max_input_gen_threshold =  2700 #3000
mode = "rnn1-unpadding/direct-gender-noun"

In [45]:
#load pickles

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))



noun_dict1 :  {'this woman': 2901, 'her': 6753, 'this girl': 2928, 'my sister': 2631, 'my wife': 2784, 'my aunt': 2781, 'She': 2571, 'my girlfriend': 2694, 'my mother': 2666, 'my daughter': 2601}
noun_dict2 :  {'this man': 2877, 'him': 4094, 'this boy': 2930, 'my brother': 2625, 'my husband': 2807, 'my uncle': 2806, 'He': 2586, 'my boyfriend': 2701, 'my father': 2652, 'my son': 2566}
noun_error1 :  {'this girl': 283, 'my wife': 386, 'my aunt': 402, 'my girlfriend': 217, 'my mother': 868, 'her': 1528, 'this woman': 485, 'my sister': 210, 'She': 238, 'my daughter': 776}
noun_error2 :  {'this boy': 268, 'my husband': 348, 'my uncle': 434, 'my boyfriend': 172, 'him': 801, 'my father': 855, 'my brother': 232, 'He': 257, 'this man': 430, 'my son': 728}


In [46]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [47]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

801

In [48]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'this girl': 283, 'my wife': 386, 'my aunt': 402, 'my girlfriend': 217, 'my mother': 868, 'this woman': 485, 'my sister': 210, 'She': 238, 'my daughter': 776}
noun_dict1: {'this woman': 2901, 'this girl': 2928, 'my sister': 2631, 'my wife': 2784, 'my aunt': 2781, 'She': 2571, 'my girlfriend': 2694, 'my mother': 2666, 'my daughter': 2601}
noun_error2: {'this boy': 268, 'my husband': 348, 'my uncle': 434, 'my boyfriend': 172, 'my father': 855, 'my brother': 232, 'He': 257, 'this man': 430, 'my son': 728}
noun_dict2: {'this man': 2877, 'this boy': 2930, 'my brother': 2625, 'my husband': 2807, 'my uncle': 2806, 'He': 2586, 'my boyfriend': 2701, 'my father': 2652, 'my son': 2566}


In [49]:
generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 2700 unique inputs generated
pred_err_count:  2831
pred_error_rate:  0.013480952380952382
********************
fairness_err_count:  893
fairness_error_rate:  0.004252380952380953
********************
Final Unique input1s: 1183
Final Unique input2s: 1201
Final Unique input pairs: 2700
********************
Final Unique PREDICTION errors: 1647
Final Unique FAIRNESS violations: 493
Final length of retrain_dict: 281
********************
noun_dict1:  {'this woman': 3336, 'this girl': 3207, 'my sister': 2851, 'my wife': 3150, 'my aunt': 3167, 'She': 2757, 'my girlfriend': 2933, 'my mother': 3613, 'my daughter': 3399, 'her': 1599}
noun_dict2:  {'this man': 3251, 'this boy': 3204, 'my brother': 2820, 'my husband': 3100, 'my uncle': 3281, 'He': 2806, 'my boyfriend': 2946, 'my father': 3575, 'my son': 3448, 'him': 627}
noun_error1:  {'this girl': 314, 'my wife': 451, 'my aunt': 468, 'my girlfriend': 267, 'my mother': 1110, 'this woman': 593, 'my sister'

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [50]:
noun_choice =  1 #Noun /Pronoun

In [51]:
err_count = 0
ITERS = 30000
num_iter = 5000 
max_input_gen_threshold =  3000
mode = "rnn1-unpadding/random-gender-noun"

In [52]:
#load pickles 

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

#bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))

noun_dict1 :  {'my aunt': 421, 'her': 988, 'my girlfriend': 472, 'this woman': 365, 'my sister': 382, 'my mom': 312, 'She': 376, 'my daughter': 403, 'this girl': 373, 'my wife': 416, 'my mother': 344}
noun_dict2 :  {'my dad': 377, 'him': 659, 'this man': 328, 'this boy': 397, 'my father': 348, 'my husband': 361, 'my boyfriend': 388, 'my brother': 410, 'my son': 400, 'my uncle': 460, 'He': 380}
noun_error1 :  {'my aunt': 94, 'my girlfriend': 56, 'her': 177, 'my mom': 54, 'this woman': 52, 'this girl': 25, 'She': 43, 'my daughter': 85, 'my wife': 65, 'my sister': 36, 'my mother': 69}
noun_error2 :  {'my dad': 39, 'this man': 47, 'him': 137, 'my boyfriend': 53, 'this boy': 41, 'my uncle': 83, 'my father': 79, 'my brother': 39, 'my husband': 48, 'my son': 64, 'He': 57}


In [53]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [54]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

137

In [55]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'my aunt': 94, 'my girlfriend': 56, 'my mom': 54, 'this woman': 52, 'this girl': 25, 'She': 43, 'my daughter': 85, 'my wife': 65, 'my sister': 36, 'my mother': 69}
noun_dict1: {'my aunt': 421, 'my girlfriend': 472, 'this woman': 365, 'my sister': 382, 'my mom': 312, 'She': 376, 'my daughter': 403, 'this girl': 373, 'my wife': 416, 'my mother': 344}
noun_error2: {'my dad': 39, 'this man': 47, 'my boyfriend': 53, 'this boy': 41, 'my uncle': 83, 'my father': 79, 'my brother': 39, 'my husband': 48, 'my son': 64, 'He': 57}
noun_dict2: {'my dad': 377, 'this man': 328, 'this boy': 397, 'my father': 348, 'my husband': 361, 'my boyfriend': 388, 'my brother': 410, 'my son': 400, 'my uncle': 460, 'He': 380}


In [56]:
generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 3002 unique inputs generated
pred_err_count:  2746
pred_error_rate:  0.013076190476190476
********************
fairness_err_count:  783
fairness_error_rate:  0.0037285714285714287
********************
Final Unique input1s: 1348
Final Unique input2s: 1369
Final Unique input pairs: 3002
********************
Final Unique PREDICTION errors: 1749
Final Unique FAIRNESS violations: 470
Final length of retrain_dict: 294
********************
noun_dict1:  {'my aunt': 905, 'my girlfriend': 859, 'this woman': 674, 'my sister': 641, 'my mom': 781, 'She': 634, 'my daughter': 980, 'this girl': 597, 'my wife': 911, 'my mother': 893, 'her': 1200}
noun_dict2:  {'my dad': 643, 'this man': 709, 'this boy': 624, 'my father': 977, 'my husband': 803, 'my boyfriend': 715, 'my brother': 679, 'my son': 882, 'my uncle': 1008, 'He': 808, 'him': 663}
noun_error1:  {'my aunt': 176, 'my girlfriend': 111, 'my mom': 128, 'this woman': 106, 'this girl': 46, 'She': 65, 'my dau

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [57]:
noun_choice =  2 #Noun /Pronoun

In [58]:
err_count = 0
ITERS = 30000
num_iter = 5000 
max_input_gen_threshold =  3000
mode = "rnn1-unpadding/gender-occupation-noun"

In [59]:
#load pickles 

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

#bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))

noun_dict1 :  {'the baker': 176, 'her': 924, 'the editor': 184, 'the attendant': 161, 'the counselor': 159, 'the hairdresser': 130, 'the secretary': 204, 'the teacher': 349, 'the receptionist': 117, 'the clerk': 152, 'the writer': 191, 'the librarian': 119, 'the auditor': 129, 'the nurse': 166, 'the the librarian': 168, 'the assistant': 142, 'the the paralegal': 125, 'the housekeeper': 172, 'the the nurse': 90, 'the cleaner': 137, 'the tailor': 183, 'the designer': 123, 'the cashier': 115}
noun_dict2 :  {'the cook': 138, 'him': 569, 'the farmer': 135, 'the physician': 147, 'the developer': 108, 'the mechanic': 127, 'the CEO': 183, 'the accountant': 139, 'the mover': 183, 'the analyst': 224, 'the driver': 193, 'the chief': 176, 'the engineer': 176, 'the sheriff': 97, 'the manager': 182, 'the salesperson': 143, 'the supervisor': 143, 'the laborer': 166, 'the lawyer': 151, 'the guard': 145, 'the janitor': 138, 'the carpenter': 125, 'the construction worker': 125, 'the technician': 154}
no

In [60]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [61]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

99

In [62]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'the secretary': 31, 'the receptionist': 36, 'the clerk': 33, 'the auditor': 25, 'the assistant': 6, 'the nurse': 18, 'the the paralegal': 16, 'the writer': 31, 'the librarian': 27, 'the the nurse': 11, 'the editor': 27, 'the the librarian': 29, 'the baker': 22, 'the teacher': 66, 'the housekeeper': 37, 'the attendant': 9, 'the designer': 19, 'the cleaner': 23, 'the tailor': 31, 'the hairdresser': 33, 'the cashier': 20, 'the counselor': 14}
noun_dict1: {'the baker': 176, 'the editor': 184, 'the attendant': 161, 'the counselor': 159, 'the hairdresser': 130, 'the secretary': 204, 'the teacher': 349, 'the receptionist': 117, 'the clerk': 152, 'the writer': 191, 'the librarian': 119, 'the auditor': 129, 'the nurse': 166, 'the the librarian': 168, 'the assistant': 142, 'the the paralegal': 125, 'the housekeeper': 172, 'the the nurse': 90, 'the cleaner': 137, 'the tailor': 183, 'the designer': 123, 'the cashier': 115}
noun_error2: {'the CEO': 19, 'the accountant': 23, 'the anal

In [63]:
generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 3002 unique inputs generated
pred_err_count:  2449
pred_error_rate:  0.011661904761904762
********************
fairness_err_count:  816
fairness_error_rate:  0.0038857142857142857
********************
Final Unique input1s: 1821
Final Unique input2s: 1848
Final Unique input pairs: 3002
********************
Final Unique PREDICTION errors: 1748
Final Unique FAIRNESS violations: 559
Final length of retrain_dict: 432
********************
noun_dict1:  {'the baker': 375, 'the editor': 277, 'the attendant': 222, 'the counselor': 246, 'the hairdresser': 378, 'the secretary': 432, 'the teacher': 502, 'the receptionist': 381, 'the clerk': 376, 'the writer': 360, 'the librarian': 340, 'the auditor': 290, 'the nurse': 263, 'the the librarian': 358, 'the assistant': 171, 'the the paralegal': 267, 'the housekeeper': 361, 'the the nurse': 186, 'the cleaner': 308, 'the tailor': 371, 'the designer': 299, 'the cashier': 258, 'her': 733}
noun_dict2:  {'the cook'

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [64]:
noun_choice =  3 #Noun /Pronoun

In [65]:
err_count = 0
ITERS = 30000
num_iter = 5000 
max_input_gen_threshold =  3000
mode = "rnn1-unpadding/gender-name-noun"

In [66]:
#load pickles 

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

#bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))

noun_dict1 :  {'Deborah': 136, 'her': 570, 'Karen': 83, 'Lisa': 129, 'Sharon': 97, 'Margaret': 122, 'Jessica': 118, 'Cynthia': 148, 'Stephanie': 114, 'Jennifer': 112, 'Donna': 94, 'Melissa': 130, 'Michelle': 97, 'Kimberly': 138, 'Sandra': 163, 'Susan': 122, 'Betty': 59, 'Rebecca': 117, 'Ashley': 123, 'Elizabeth': 93, 'Mary': 137, 'Dorothy ': 92, 'Nancy': 74, 'Barbara': 150, 'Patricia': 130, 'Emily': 116, 'Laura': 105, 'Linda': 124, 'Sarah': 137, 'Amanda': 86, 'Carol': 88}
noun_dict2 :  {'Andrew': 139, 'Kenneth': 132, 'Richard': 167, 'him': 561, 'Donald': 112, 'John ': 135, 'Robert ': 92, 'Anthony': 107, 'Daniel': 117, 'Joshua': 118, 'Jason': 92, 'Steven': 117, 'William ': 142, 'George': 84, 'Edward': 74, 'Matthew': 122, 'Ryan': 177, 'Christopher': 137, 'Kevin': 88, 'Paul': 122, 'Joseph': 97, 'Brian': 79, 'Michael ': 111, 'Mark': 113, 'Jeffrey': 145, 'James': 112, 'David ': 93, 'Thomas': 85, 'Charles': 90, 'Ronald': 110, 'Timothy': 134}
noun_error1 :  {'Karen': 24, 'her': 90, 'Sharon': 

In [67]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [68]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

100

In [69]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'Karen': 24, 'Sharon': 10, 'Margaret': 12, 'Stephanie': 22, 'Jennifer': 19, 'Donna': 24, 'Deborah': 17, 'Melissa': 17, 'Michelle': 19, 'Lisa': 26, 'Kimberly': 20, 'Jessica': 19, 'Rebecca': 35, 'Elizabeth': 16, 'Mary': 20, 'Nancy': 17, 'Betty': 5, 'Emily': 24, 'Sandra': 31, 'Barbara': 17, 'Laura': 27, 'Cynthia': 24, 'Sarah': 33, 'Patricia': 11, 'Carol': 15, 'Susan': 22, 'Ashley': 10, 'Linda': 26, 'Dorothy ': 13, 'Amanda': 4}
noun_dict1: {'Deborah': 136, 'Karen': 83, 'Lisa': 129, 'Sharon': 97, 'Margaret': 122, 'Jessica': 118, 'Cynthia': 148, 'Stephanie': 114, 'Jennifer': 112, 'Donna': 94, 'Melissa': 130, 'Michelle': 97, 'Kimberly': 138, 'Sandra': 163, 'Susan': 122, 'Betty': 59, 'Rebecca': 117, 'Ashley': 123, 'Elizabeth': 93, 'Mary': 137, 'Dorothy ': 92, 'Nancy': 74, 'Barbara': 150, 'Patricia': 130, 'Emily': 116, 'Laura': 105, 'Linda': 124, 'Sarah': 137, 'Amanda': 86, 'Carol': 88}
noun_error2: {'Donald': 26, 'John ': 24, 'Robert ': 35, 'Anthony': 21, 'Daniel': 17, 'Steven': 

In [70]:
generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 3000 unique inputs generated
pred_err_count:  2378
pred_error_rate:  0.011323809523809524
********************
fairness_err_count:  717
fairness_error_rate:  0.0034142857142857143
********************
Final Unique input1s: 1983
Final Unique input2s: 1982
Final Unique input pairs: 3000
********************
Final Unique PREDICTION errors: 1720
Final Unique FAIRNESS violations: 504
Final length of retrain_dict: 406
********************
noun_dict1:  {'Deborah': 202, 'Karen': 280, 'Lisa': 274, 'Sharon': 158, 'Margaret': 194, 'Jessica': 266, 'Cynthia': 251, 'Stephanie': 249, 'Jennifer': 262, 'Donna': 290, 'Melissa': 198, 'Michelle': 204, 'Kimberly': 191, 'Sandra': 333, 'Susan': 288, 'Betty': 136, 'Rebecca': 307, 'Ashley': 159, 'Elizabeth': 169, 'Mary': 220, 'Dorothy ': 188, 'Nancy': 259, 'Barbara': 208, 'Patricia': 175, 'Emily': 246, 'Laura': 298, 'Linda': 322, 'Sarah': 317, 'Amanda': 122, 'Carol': 147, 'her': 553}
noun_dict2:  {'Andrew': 351, 'Ken

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [71]:
noun_choice =  5 #Noun /Pronoun

In [72]:
err_count = 0
ITERS = 30000
num_iter = 5000 
max_input_gen_threshold =  3000
mode = "rnn1-unpadding/racial-name-noun"

In [73]:
#load pickles 

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

#bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))

noun_dict1 :  {'Andrew': 183, 'Josh': 207, 'her': 804, 'Melanie': 173, 'Amanda': 206, 'Stephanie': 180, 'Roger': 146, 'Heather': 198, 'Ellen': 203, 'Justin': 199, 'Adam': 141, 'Nancy': 157, 'Betsy': 193, 'Courtney': 138, 'Jack': 168, 'Ryan': 197, 'Frank': 153, 'Kristin': 146, 'Alan': 227, 'Harry': 215, 'Katie': 101}
noun_dict2 :  {'Lamar': 135, 'him': 566, 'Alonzo': 168, 'Shaniqua': 125, 'Tanisha': 164, 'Tia': 198, 'Jamel': 107, 'Leroy': 214, 'Darnell': 167, 'Malik': 126, 'Jasmine': 252, 'Terrence': 102, 'Alphonse': 163, 'Latoya': 147, 'Jerome': 266, 'Lakisha': 260, 'Latisha': 150, 'Ebony': 205, 'Torrance': 211, 'Shereen': 205, 'Nichelle': 206}
noun_error1 :  {'Andrew': 41, 'Josh': 32, 'her': 140, 'Melanie': 32, 'Roger': 28, 'Ellen': 40, 'Justin': 36, 'Nancy': 58, 'Courtney': 25, 'Heather': 33, 'Stephanie': 32, 'Kristin': 11, 'Adam': 14, 'Frank': 24, 'Jack': 17, 'Ryan': 18, 'Betsy': 12, 'Alan': 27, 'Harry': 52, 'Amanda': 29, 'Katie': 28}
noun_error2 :  {'Lamar': 12, 'Alonzo': 45, 'him'

In [74]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [75]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

117

In [76]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'Andrew': 41, 'Josh': 32, 'Melanie': 32, 'Roger': 28, 'Ellen': 40, 'Justin': 36, 'Nancy': 58, 'Courtney': 25, 'Heather': 33, 'Stephanie': 32, 'Kristin': 11, 'Adam': 14, 'Frank': 24, 'Jack': 17, 'Ryan': 18, 'Betsy': 12, 'Alan': 27, 'Harry': 52, 'Amanda': 29, 'Katie': 28}
noun_dict1: {'Andrew': 183, 'Josh': 207, 'Melanie': 173, 'Amanda': 206, 'Stephanie': 180, 'Roger': 146, 'Heather': 198, 'Ellen': 203, 'Justin': 199, 'Adam': 141, 'Nancy': 157, 'Betsy': 193, 'Courtney': 138, 'Jack': 168, 'Ryan': 197, 'Frank': 153, 'Kristin': 146, 'Alan': 227, 'Harry': 215, 'Katie': 101}
noun_error2: {'Lamar': 12, 'Alonzo': 45, 'Shaniqua': 14, 'Jamel': 13, 'Darnell': 44, 'Malik': 10, 'Tia': 21, 'Latoya': 15, 'Jasmine': 60, 'Jerome': 85, 'Alphonse': 23, 'Lakisha': 33, 'Leroy': 24, 'Torrance': 53, 'Shereen': 44, 'Ebony': 11, 'Terrence': 6, 'Nichelle': 27, 'Tanisha': 27, 'Latisha': 12}
noun_dict2: {'Lamar': 135, 'Alonzo': 168, 'Shaniqua': 125, 'Tanisha': 164, 'Tia': 198, 'Jamel': 107, 'Leroy': 

In [77]:
generate_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 3003 unique inputs generated
pred_err_count:  2630
pred_error_rate:  0.012523809523809524
********************
fairness_err_count:  787
fairness_error_rate:  0.003747619047619048
********************
Final Unique input1s: 1745
Final Unique input2s: 1745
Final Unique input pairs: 3003
********************
Final Unique PREDICTION errors: 1818
Final Unique FAIRNESS violations: 533
Final length of retrain_dict: 377
********************
noun_dict1:  {'Andrew': 409, 'Josh': 304, 'Melanie': 383, 'Amanda': 388, 'Stephanie': 372, 'Roger': 336, 'Heather': 443, 'Ellen': 431, 'Justin': 349, 'Adam': 247, 'Nancy': 525, 'Betsy': 261, 'Courtney': 338, 'Jack': 267, 'Ryan': 349, 'Frank': 310, 'Kristin': 210, 'Alan': 295, 'Harry': 497, 'Katie': 391, 'her': 850}
noun_dict2:  {'Lamar': 245, 'Alonzo': 429, 'Shaniqua': 255, 'Tanisha': 384, 'Tia': 303, 'Jamel': 283, 'Leroy': 346, 'Darnell': 448, 'Malik': 223, 'Jasmine': 455, 'Terrence': 143, 'Alphonse': 338, 'Latoya

### F. Test for Neutral (Sentiment) Sentences

In [78]:
err_count = 0
ITERS = 30000
num_iter = 5000 
max_input_gen_threshold =  3000
mode = "rnn1-unpadding/neutral-sentiments"

In [79]:
#load pickles 

pred_err_count, fairness_err_count = 0, 0

unique_pred_input1_error_set, unique_fairness_input1_error_set = set(), set() 
retrain_dict = dict()

count_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
pred_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]
fairness_error_dict = [{}, {}, {}, {}, {}, {}, {}, {}]

#bias_pair_count, bias_pair_pred_error, bias_pair_fairness_error = {}, {}, {}

bias_pair_count = [{}, {}, {}, {}, {}, {}]
bias_pair_pred_error = [{}, {}, {}, {}, {}, {}]
bias_pair_fairness_error = [{}, {}, {}, {}, {}, {}]
noun_dict1, noun_dict2, noun_error1, noun_error2 = {}, {}, {}, {}

c_vals = [noun_dict1, noun_dict2, noun_error1, noun_error2, unique_pred_input1_error_set, pred_err_count, unique_fairness_input1_error_set, fairness_err_count, retrain_dict, pred_error_dict, fairness_error_dict, bias_pair_pred_error, bias_pair_fairness_error]
c_names = ["noun_dict1", "noun_dict2", "noun_error1", "noun_error2", "unique_pred_input1_error_set", "pred_err_count", "unique_fairness_input1_error_set", "fairness_err_count", "retrain_dict", "pred_error_dict", "fairness_error_dict", "bias_pair_pred_error", "bias_pair_fairness_error"]

assert len(c_names) == len(c_vals), \
    "ERROR: bug in variables names for stored inputs {}, {}".format(c_names)

for i in range(0, len(c_vals)):
    store = c_names[i] + ".pickle"
    target = '../Exploration/saved_pickles/exploration/' + mode + '/' + store
    if os.path.exists(target):
        if os.path.getsize(target) > 0:
#             print(os.path.getsize(target))
#             print(os.path.exists(target))
            with open('../Exploration/saved_pickles/exploration/' + mode + '/' + store, 'rb') as handle:
                c_vals[i] = pickle.load(handle)
                if i <4:
                    print(c_names[i], ": ", c_vals[i])
#         else:
#             print("ERROR {} is empty".format(target))
#     else:
#         print("ERROR {} does not exist".format(target))

noun_dict1 :  {'my wife': 19068, 'my sister': 18873, 'my girlfriend': 19056, 'my aunt': 19263, 'She': 18939, 'my mother': 18576, 'her': 18840, 'my daughter': 18738, 'this girl': 18948, 'this woman': 19410, 'my mom': 9129, 'the tailor': 126, 'the hairdresser': 99, 'the counselor': 114, 'the housekeeper': 111, 'the teacher': 264, 'the secretary': 117, 'the the paralegal': 138, 'the cashier': 96, 'the the librarian': 99, 'the cleaner': 111, 'the designer': 126, 'the the nurse': 102, 'the auditor': 99, 'the receptionist': 84, 'the attendant': 117, 'the baker': 102, 'the assistant': 120, 'the nurse': 81, 'the clerk': 123, 'the writer': 123, 'the librarian': 135, 'the editor': 111}
noun_dict2 :  {'my husband': 18834, 'my brother': 19047, 'my boyfriend': 18876, 'my uncle': 19230, 'He': 19002, 'my father': 18513, 'my son': 18792, 'this boy': 19194, 'this man': 19413, 'my dad': 9099, 'the chief': 99, 'the laborer': 120, 'the carpenter': 135, 'the salesperson': 126, 'the physician': 114, 'the te

In [80]:
noun_dict1 = c_vals[0]
noun_dict2 = c_vals[1]
noun_error1 =  c_vals[2]
noun_error2 =  c_vals[3]

In [81]:
noun_dict1.pop('her', None)
noun_error1.pop('her', None)
noun_dict2.pop('him', None)
noun_error2.pop('him', None)

In [82]:
print("noun_error1: {}".format(noun_error1))
print("noun_dict1: {}".format(noun_dict1))
print("noun_error2: {}".format(noun_error2))
print("noun_dict2: {}".format(noun_dict2))

noun_error1: {'my sister': 3269, 'She': 4194, 'my mother': 13304, 'my daughter': 9321, 'my aunt': 3159, 'my girlfriend': 1756, 'this girl': 918, 'my wife': 880, 'this woman': 879, 'my mom': 936, 'the hairdresser': 69, 'the counselor': 19, 'the housekeeper': 37, 'the tailor': 17, 'the teacher': 81, 'the the librarian': 32, 'the cashier': 22, 'the secretary': 43, 'the designer': 13, 'the receptionist': 59, 'the baker': 20, 'the the paralegal': 10, 'the nurse': 8, 'the writer': 11, 'the auditor': 35, 'the cleaner': 36, 'the clerk': 50, 'the librarian': 41, 'the assistant': 18, 'the attendant': 20, 'the the nurse': 17, 'the editor': 19}
noun_dict1: {'my wife': 19068, 'my sister': 18873, 'my girlfriend': 19056, 'my aunt': 19263, 'She': 18939, 'my mother': 18576, 'my daughter': 18738, 'this girl': 18948, 'this woman': 19410, 'my mom': 9129, 'the tailor': 126, 'the hairdresser': 99, 'the counselor': 114, 'the housekeeper': 111, 'the teacher': 264, 'the secretary': 117, 'the the paralegal': 13

In [83]:
generate_neutral_tests_probabilistically(noun_choice, ITERS, max_input_gen_threshold, mode, noun_error1, noun_error2, noun_dict1, noun_dict2)

Maximum input generation threshold reached, 3002 unique inputs generated
Maximum input generation threshold reached, 3002 unique inputs generated
pred_err_count:  4395
pred_error_rate:  0.02092857142857143
********************
fairness_err_count:  1714
fairness_error_rate:  0.008161904761904763
********************
Final Unique input1s: 310
Final Unique input2s: 329
Final Unique input pairs: 3002
********************
Final Unique PREDICTION errors: 3002
Final Unique FAIRNESS violations: 1056
Final length of retrain_dict: 244
********************
noun_dict1:  {'my wife': 19083, 'my sister': 18942, 'my girlfriend': 19113, 'my aunt': 19356, 'She': 19062, 'my mother': 19029, 'my daughter': 18975, 'this girl': 18969, 'this woman': 19419, 'my mom': 9171, 'the tailor': 207, 'the hairdresser': 510, 'the counselor': 204, 'the housekeeper': 258, 'the teacher': 423, 'the secretary': 306, 'the the paralegal': 186, 'the cashier': 228, 'the the librarian': 285, 'the cleaner': 291, 'the designer': 21